## a-AlphaBio homework 
### misc. futzing about.... prob messy

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
# import libraries
import numpy as np
import pickle as pk
import pandas as pd
import math
import os
import yaml
import sys
import pytorch_lightning as pl
import torch
# import sys
# sys.path.append("../")

import matplotlib.pyplot as plt
%matplotlib inline

import torch

In [ ]:
# multiply two 1d tensors to get a 2d tensor
def outer_product(t1, t2):
    return t1[:, None] * t2[None, :]
#     return torch.einsum('i,j->ij', t1, t2)

In [ ]:
x = torch.randn(5)
y = torch.randn(5)

# vx = x - torch.mean(x)
# vy = y - torch.mean(y)
# c = outer_product(vx, vy)
# c = c/math.sqrt(torch.norm(vx)*torch.norm(vy))

# create a 2x5 matrix c from x and y
c = torch.stack([x, y])

print(c)

p = torch.corrcoef(c)
print(p)


In [ ]:


print(a)
a = a.view(1, -1)   #a.unsqueeze(0)
# print(a.shape)
b = b.view(-1, 1)  #.unsqueeze(-1)
# print(b.shape)

# print(a)
# print(b)

c = a*b

torch.corrcoef(c)

# torch.einsum('i,j->ij', a, b)

# outer_product(a, b)

In [ ]:
a = 'QVQLVQSGVEVKKPGASVKVSCKASGYTFTNYYWVRQAPGQGLEWMGGINPSNGGTNFNEKFKNRVTLTTDSSTTTAYMELKSLQFDDTAVYYCARRDYRFDMGFDYWYWGQGTTVTV'
l = 'SSGGGGSGGGGSGGGGSE'
b = 'IVLTQSPATLSLSPGERATLSCRASKGVSTSGYSYLHWYQQKPGQAPRLLIYLASYLESGVPARFSGSGSGTDFTLTISSLEPEDFAVYYCQHSRDLPLTFGGGTKVEIK'

print(len(a), len(l), len(b))

In [ ]:
import numpy as np, pandas as pd
from torch import tensor, nn

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=1, keepdims=True)

labels = [0, 2, 1, 3]
logits =  np.array([
    [3.5, -3.45, 0.23, 1.25],
    [-2.14, 0.54, 2.67, -5.23],
    [-1.34, 5.01, -1.54, -1.17],
    [ -2.98, -1.37, 1.54,5.23]
])
probs = softmax(logits)
print(probs)
log_probs = np.log(softmax(logits))
print('\n', log_probs)

nll = -(log_probs[range(len(labels)), labels])
nll

In [ ]:
labels = torch.tensor([0, 2, 1, 3])
logits =  torch.tensor(np.array([
    [3.5, -3.45, 0.23, 1.25],
    [-2.14, 0.54, 2.67, -5.23],
    [-1.34, 5.01, -1.54, -1.17],
    [ -2.98, -1.37, 1.54,5.23]
]))

output = F.cross_entropy(logits, labels, reduction='none')
print(output.shape)
loss = torch.mean(output) #output.sum() / mask.shape[0]
print(loss)
print()
loss = output.sum() / labels.shape[0]
print(loss)
print()

lossfn = torch.nn.CrossEntropyLoss()
output = lossfn(logits, labels)
print(output)

In [ ]:
import torch.nn.functional as F

# loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print('input shape:', input.shape)
print('target shape:', target.shape)
print(input)
print(target)

output = F.cross_entropy(input, target, reduction='none')
print(output.shape)
loss = torch.mean(output) #output.sum() / mask.shape[0]
print(loss)
print()

lossfn = torch.nn.CrossEntropyLoss()
output = lossfn(input, target)
print(output)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(output)

In [ ]:
x = torch.randn(4, 4)
print(x.size())
y = x.view(16)
print(y.size())
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(z.size())
a = x.view(-1)
print(a.size())

In [ ]:
q = torch.tensor([0.1])
p = torch.tensor([1])

ce = - torch.sum(p * torch.log(q))
print(ce)

In [ ]:
a = 'TNYYMYWVRQAPGQGLEWMGGINPSNGGTNFNEKFKNRVTLTTDSSTTTAYMELKSLQFDDTAVYYCARRDYRFDMGFD'
print(len(a))

In [ ]:
# Read the config
config_path = './config/tform_params.yaml'
with open(config_path, 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

model_config = config['model_params']
train_config = config['train_params']    

print(model_config)
print(train_config)
pl.seed_everything(config['seed'])


In [ ]:
import math
import random
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class scFv_diy_pretrain_Dataset(Dataset):
    """
    Dataset class for the paired sequences for pretraining the model
    The data consists of paired heavy and light chain sequences
    They are concatenated with a "standard" scFv linker sequence
    Hence the "DIY" in the name
    50% of the time, the heavy and light chains are swapped

    The dataset is a csv file with the following columns:
    - sequence_alignment_aa_heavy: amino acid sequence of the heavy chain
    - sequence_alignment_aa_light: amino acid sequence of the light chain

    The dataset is used for pretraining the model with the following task:
    Masked language model training
        The masked language model training sequence with a small percentage of the amino acids masked
        sequence will look something like:           [CLS, aa, aa, MASK, aa, MASK, aa, ...PAD,..PAD]
        corresponding mask will look something like: [0,   0,   0,   aa,  0,   aa, 0, ...0, 0]

    """
    def __init__(self, config, block_size, csv_file_path, skiprows=0, inference=False, regularize=False):  
        super().__init__()
        print('DIY scFv sequences created by this dataset class')
        self.config = config
        self.block_size = block_size
        self.inference = inference
        self.regularize = regularize # sequence flipping etc...
        assert config['train_type'] in ['mask_lang_model', 'regression'], 'train_type must be either "mask_lang_model" or "regression"'
        self.train_type = config['train_type']

        print('reading the data from:', csv_file_path)
        self.df = pd.read_csv(csv_file_path, skiprows=skiprows)
        
        # 20 amino acids + special tokens (CLS, X, PAD, MASK, SEP) 
        self.chars = ['CLS', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 
                      'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X', 'MASK', 'PAD', 'SEP']
        self.first_aa_idx = 1
        self.last_aa_idx = len(self.chars) - 4
        print('vocabulary:', self.chars)
        data_size, vocab_size = self.df.shape[0], len(self.chars)
        print('data has %d rows, %d vocab size' % (data_size, vocab_size))

        # encoding and decoding residues
        self.stoi = { ch:i for i,ch in enumerate(self.chars) }
        self.itos = { i:ch for i,ch in enumerate(self.chars) }
        self.vocab_size = vocab_size

        self.linker = 'SSGGGGSGGGGSGGGGSE' # fixed linker sequnce
        print('Using fixed linker sequence:', self.linker)

    #-------------------------------------------------------
    # Create the mask for masked language model training
    # and the corresponding modified dix sequence
    #
    # if cls = True, prepend a CLS token, else don't
    #-------------------------------------------------------
    def create_mask(self, dix, cls=True):

        # prepend the CLS token to the sequence
        if cls:
          dix = torch.cat((torch.tensor([self.stoi['CLS']], dtype=torch.long), dix))
          
        mask = torch.zeros_like(dix)

        # training will be masked language model
        if self.train_type == 'mask_lang_model':

            # get number of tokens to mask
            n_pred = max(1, int(round(self.block_size * self.config['mask_prob'])))

            # indices of the tokens that will be masked (a random permutation of n_pred of the tokens)
            masked_idx = torch.randperm(len(dix)-1, dtype=torch.long)[:n_pred]
            masked_idx += 1  # so we never mask the CLS token
            mask = torch.zeros_like(dix)

            # copy the actual tokens to be masked, to the mask
            mask[masked_idx] = dix[masked_idx]
            # And replace all masked tokens with the MASK token
            dix[masked_idx] = self.stoi['MASK']

        assert(dix.shape[0] == mask.shape[0]), 'dix and mask shape is not equal'
        return dix, mask


    def get_vocab_size(self):
        return self.vocab_size

    def get_block_size(self):
        return self.block_size

    def __len__(self):
        return self.df.shape[0] 

    #-------------------------------------------------------
    # Get the sequence, and mask data for the given index
    #-------------------------------------------------------
    def __getitem__(self, idx):
        seq1 = self.df.loc[idx, 'sequence_alignment_aa_heavy']
        seq2 = self.df.loc[idx, 'sequence_alignment_aa_light']

        assert isinstance(seq1, str), 'seq1 is not a string'
        assert isinstance(seq2, str), 'seq2 is not a string'

        # flip the order of heavy, light chains 50% of the time
        if random.random() > 0.5:
            seq1, seq2 = seq2, seq1

        chunk = seq1 + self.linker + seq2
        print('chunk shape:', len(chunk))

        # hardwired for now
        Kd = 0
        name = 'none'

        # trim if necessary
        if len(chunk) >= (self.block_size-1):  # '-1' is to accomodate the CLS tokens below
            chunk = chunk[0:self.block_size-1]
            print('trimmed chunk shape:', len(chunk))

        # encode the string chunk
        dix = torch.tensor([self.stoi[s] for s in chunk], dtype=torch.long)
        print('dix:', dix)

        # create the mask for the masked language model training
        # and the corresponding modified dix sequence
        dix, mask = self.create_mask(dix, cls=True)

        # Append with PAD tokens if necessary
        if dix.shape[0] < self.block_size:
            dix = torch.cat((dix, torch.tensor([self.stoi['PAD']] * (self.block_size - len(dix)), dtype=torch.long)))
            mask = torch.cat((mask, torch.tensor([0] * (self.block_size - len(mask)), dtype=torch.long)))

        assert(dix.shape[0] == self.block_size), 'dix shape is not equal to block_size'
        assert(mask.shape[0] == self.block_size), 'mask shape is not equal to block_size'

        return dix, mask, torch.tensor([Kd], dtype=torch.float32), name


In [ ]:
csv_file_path = './data/oas_2/paired/paired_1_set.csv'
dataset = scFv_diy_pretrain_Dataset(train_config, model_config['block_size'], csv_file_path, regularize=train_config['sequence_regularize'])

x, mask, kd, name = dataset.__getitem__(0)
print('x shape:', x.shape, 'mask shape:', mask.shape, ', kd:', kd, ', name:', name)

print('x[-10:]:', x[-10:])
print()

print(x)
print()
print(mask)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset, batch_size=256) #config['batch_size'])
it = iter(train_loader)
x, mask, kd, _ = next(it)
print('x.dtype:', x.dtype, ', x.shape:', x.shape, ', mask.shape:', mask.shape, ', kd.shape:', kd.shape)

In [ ]:
import math
import random
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
# from datasets.scFv_dataset import scFv_Dataset

#--------------------------------------------------------
# Simple wrapper Dataset to turn output from the scFv dataset
# into a B&W image for use in a CNN model
#--------------------------------------------------------
class CNN_Dataset_BGR(Dataset):
    """
    Emits 2D B&W images and binding energies
    """
    def __init__(self, config, csv_file_path, transform=None, skiprows=0, inference=False):  
        super().__init__()
        self.scFv_dataset = scFv_Dataset(config, csv_file_path, skiprows, inference)
        self.config = config
        self.img_shape = config['image_shape']
        self.transform = transform
         
        chars = self.scFv_dataset.chars
        groups= ['none', 'nonpolar', 'nonpolar', 'neg', 'neg', 'nonpolar', 'nonpolar', 'pos', 'nonpolar', 'pos', 'nonpolar', 'nonpolar', 'neg', 
                'nonpolar', 'neg', 'pos', 'polar', 'polar', 'nonpolar', 'nonpolar', 'polar', 'none', 'none', 'none']
        
        # for VIT, since the residue encodings are spread over 8-bits, assign encodings to groups that spread across the 8-bits
        group_encodings = { 'none'    : int('11001100', base=2), 
                            'polar'   : int('00110011', base=2),
                            'nonpolar': int('01100110', base=2), 
                            'pos'     : int('01010101', base=2),
                            'neg'     : int('10101010', base=2)} 
        
        print('group_encodings:', group_encodings)

        # map encoded sequence to groups
        self.i_to_grp = {self.scFv_dataset.stoi[ch]:group_encodings[i] for ch,i in zip(chars, groups)} 

        # The relative mutation frequence for each amino acid position in the scFv sequences over the entire clean_3 dataset
        # This fixed-array is 241 elements long. (I clipped off the last 5 residues from the 246 residue sequences for the VIT model)
        self.rel_mutation_freq = torch.tensor([ 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.7778,
                                                0.9444, 0.9444, 1.0000, 1.0000, 1.0000, 0.9444, 1.0000, 1.0000, 0.8889,
                                                0.5556, 0.5000, 0.2222, 0.3333, 0.2778, 0.6111, 0.4444, 0.5556, 1.0000,
                                                0.8333, 0.8333, 0.8333, 0.9444, 0.7778, 0.8333, 0.6111, 1.0000, 0.8889,
                                                0.3333, 0.9444, 0.8889, 0.1111, 0.3889, 0.9444, 0.2778, 0.9444, 0.8333,
                                                0.5000, 1.0000, 1.0000, 1.0000, 0.9444, 0.6111, 0.6111, 0.7778, 0.2778,
                                                0.8889, 0.3889, 0.9444, 1.0000, 0.3889, 0.9444, 1.0000, 0.9444, 0.2222,
                                                0.7778, 0.5556, 0.8889, 0.2222, 0.7778, 0.6111, 0.6667, 0.8333, 0.8333,
                                                1.0000, 1.0000, 0.8889, 0.8333, 0.8333, 0.9444, 0.7222, 0.9444, 0.9444,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                                                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]) 
        
        self.mutation_freq_encoded = self.rel_mutation_freq * 255
        self.mutation_freq_encoded = torch.ceil(self.mutation_freq_encoded).to(torch.long)
        print('min mutation freq:', torch.min(self.mutation_freq_encoded), 'max mutation freq:', torch.max(self.mutation_freq_encoded))
        print('some mutation_freq_encoded values:', self.mutation_freq_encoded[60:70])
        
    def get_vocab_size(self):
        return self.scFv_dataset.vocab_size

    def get_block_size(self):
        return self.config['block_size']

    def __len__(self):
        return self.scFv_dataset.__len__()

    def _bin(self, x):
        return format(x, '08b')

    def _encode_channel(self, x, shape):
        d = ''.join([self._bin(val) for val in x])
        d = [int(x) for x in d] # turn d into a list of integers, one for each bit
        t = torch.tensor(d[:(shape[0]*shape[1])], dtype=torch.float32) # this is for shape matrix
        t = t.reshape(shape)
        return t

    """ Returns image, Kd pairs used for CNN training """
    def __getitem__(self, idx):

        dix, kd = self.scFv_dataset.__getitem__(idx)

        # The residue encoding channel
        ch_1 = self._encode_channel(dix, self.img_shape)
        print('ch_1:', ch_1.shape, torch.min(ch_1), torch.max(ch_1))

        # The residue group encoding channel
        dix_grp = torch.tensor([self.i_to_grp[i] for i in dix.numpy().tolist()], dtype=torch.long)
        ch_2 = self._encode_channel(dix_grp, self.img_shape)
        print('ch_2:', ch_2.shape, torch.min(ch_2), torch.max(ch_2))   

        # The mutation frequency channel; anything not an amino acid gets a zero.
        ch3_in = torch.zeros_like(dix)
        # First aa is always position 1 (0 is a CLS token)
        ch3_in[1:len(self.mutation_freq_encoded)+1] = self.mutation_freq_encoded
        ch_3 = self._encode_channel(ch3_in, self.img_shape)
        print('ch_3:', ch_3.shape, torch.min(ch_3), torch.max(ch_3))   

        # stack the 3 channels into a bgr image
        bgr_img = torch.stack((ch_1, ch_2, ch_3), dim=0) * 255

        if self.transform:
            bgr_img = self.transform(bgr_img)
            
        # Normalize image [-1, 1]
        bgr_img = (bgr_img - 127.5)/127.5


        return bgr_img, kd, ch_1, ch_2, ch_3 

In [ ]:
from torch.utils.data import DataLoader
# from datasets.scFv_dataset import scFv_Dataset as dataset
from torchvision.transforms.v2 import Resize, Compose, ToDtype, RandomHorizontalFlip, RandomVerticalFlip 

# train_transforms = Compose([ToDtype(torch.float32, scale=False),
#                             RandomHorizontalFlip(p=0.25),
#                             RandomVerticalFlip(p=0.25)])

train_data_path = config['train_data_path']  
train_dataset = CNN_Dataset_BGR(config, train_data_path) #, train_transforms)
print(train_dataset.__len__())
config['vocab_size'] = train_dataset.get_vocab_size()
print('config[vocab_size]:', config['vocab_size'], ', config[block_size]:', config['block_size'])

train_loader = DataLoader(train_dataset, shuffle=True, pin_memory=True, batch_size=config['batch_size'], num_workers=config['num_workers'])


In [ ]:
img, kd, ch_1, ch_2, ch_3 = train_dataset.__getitem__(105)
print(img.dtype)
# change the order of the channels to be (H, W) instead of (C, H, W)
rgb_img = img.permute(1, 2, 0)

print('img shape:', img.shape, ', kd:', kd)
# plt.imshow(rgb_img) #, cmap='gray')

plt.imshow(ch_3, cmap='gray')

print(torch.min(rgb_img), ', ', torch.max(rgb_img), ', ', torch.mean(rgb_img), ', ', torch.std(rgb_img))


----
### Examine other data that may be added as input channels to Vision Transformer.


In [ ]:
# Classify the amino acids into their usual groups
# polar, nonpolar, positively charged, negatively charged, or none (i.e. CLS, SEP, PAD)
#
# 20 naturally occuring amino acids in human proteins plus MASK token, 
# 'X' is a special token for unknown amino acids, and CLS token is for classification, and PAD for padding
chars = ['CLS', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X', 'MASK', 'PAD']
groups= ['none', 'nonpolar', 'nonpolar', 'neg', 'neg', 'nonpolar', 'nonpolar', 'pos', 'nonpolar', 'pos', 'nonpolar', 'nonpolar', 'neg', 
         'nonpolar', 'neg', 'pos', 'polar', 'polar', 'nonpolar', 'nonpolar', 'polar', 'none', 'none', 'none']
print('\nvocabulary:', chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# for VIT, since the residue encodings are spread over 8-bits, assign encodings to groups that spread across the 8-bits
group_encodings = { 'none'    : int('00101000', base=2), 
                    'polar'   : int('00110011', base=2),
                    'nonpolar': int('11001100', base=2), 
                    'pos'     : int('01010101', base=2),
                    'neg'     : int('10101010', base=2)} 

print('group_encodings:', group_encodings)

# maps amino acid to group                     
s_to_grp = {ch:group_encodings[i] for ch,i in zip(chars, groups)} 
# maps encoded residue to group
i_to_grp = {stoi[ch]:group_encodings[i] for ch,i in zip(chars, groups)} 

print('\ns_to_grp:', s_to_grp)
print('\ni_to_grp:', i_to_grp)




In [ ]:
import torch
def _bin(x):
    return format(x, '08b')

def _encode_channel(x, shape=(48,48)):
    d = ''.join([_bin(x[i]) for i in x.numpy()])
    # turn d into a list of integers, one for each bit
    d = [int(x) for x in d]    
    t = torch.tensor(d[:(shape[0]*shape[1])], dtype=torch.float32) # this is for 46,46 matrix
    t = t.reshape(shape)
    # t = t.unsqueeze(0) # add channel dimension
    return t


In [ ]:
# Make a heat-map for the variability of each position in the sequence?
# That somehow changes with each sequence?
#